In [14]:
import os
os.chdir("../../../../")

In [90]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from openpyxl import load_workbook
import csv
import codecs
import copy
from collections import defaultdict
from scipy.spatial import distance

In [19]:
from tasks.data_loading.src.s3_client import *

In [79]:
s3_client = S3Client(creds_filepath="/Users/dafirebanks/Documents", creds_filename="credentials.json", 
              bucket_name="wri-nlp-policy", language="english")

In [80]:
s3_client

In [82]:
# Define params
init_at_doc = 0
end_at_doc = 10

i = 0
for sent_id, sent_map in s3_client.load_sentences("english", init_at_doc, end_at_doc):
    print("Sent_id:", sent_id)
    print("Text:", sent_map['text'])
    print("=======================================")
    i += 1
    if i == 4:
        break

Sent_id: 0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_0
Text: Federal Register, Volume 76 Issue 146 (Friday, July 29, 2011) [Federal Register Volume 76, Number 146 (Friday, July 29, 2011)] [Rules and Regulations] [Pages 45397-45399] From the Federal Register Online via the Government Publishing Office [[URL] [FR Doc No: 2011-19250]  ========================================================================  Rules and Regulations   Federal Register  ________________________________________________________________________    This section of the FEDERAL REGISTER contains regulatory documents  having general applicability and legal effect, most of which are keyed  to and codified in the Code of Federal Regulations, which is published  under 50 titles pursuant to 44 USC 1510.
Sent_id: 0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_1
Text:    The Code of Federal Regulations is sold by the Superintendent of Documents.
Sent_id: 0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_2
Text:  Prices of new

In [83]:
# Define queries
path = "tasks/data_augmentation/input/"
filename = "English_queries.xlsx"
file = path + filename
df = pd.read_excel(file, engine='openpyxl', sheet_name = "Hoja1", usecols = "A:C")

queries = {}
for index, row in df.iterrows():
    queries[row['Query']] = row['Policy instrument']

In [99]:
queries

{'Applicants may provide users with payments for the costs of activities consistent with the program.': 'Direct payment',
 'Given the conditions, state funding of the grants program to be increased for the 2019-20 state fiscal year and grant money to be expended to fulfill the purposes': 'Direct payment',
 'The program offers applicants incentive payments for conducting management practices': 'Direct payment',
 'joint applicants shall be considered as one eligible applicant and entitled to receive the maximum payment.': 'Direct payment',
 'We seek to fund implementation of plans that guide the strategic management': 'Direct payment',
 'Eligible applicants can receive up to $7,000 of assistance each year': 'Direct payment',
 'The board may issue a grant to fund a management plan': 'Direct payment',
 'the Fund shall be utilised for the right purpose and system of concurrent monitoring and evaluation should be evolved and implemented': 'Direct payment',
 'Additional funds for cost-share p

In [ ]:
###### TODO We may need the storage mechanism from this later
def save_results_as_separate_csv(results_dictionary, queries_dictionary, init_doc, results_limit, aws_id, aws_secret):
    path = "s3://wri-nlp-policy/english_documents/assisted_labeling"
    col_headers = ["sentence_id", "similarity_score", "text"]
    for i, query in enumerate(results_dictionary.keys()):
        filename = f"{path}/query_{queries_dictionary[query]}_{i}_results_{init_doc}.csv"
        pd.DataFrame(results_dictionary[query], columns=col_headers).head(results_limit).to_csv(filename, storage_options={"key": aws_id, "secret": aws_secret})

In [84]:
# Define params
init_at_doc = 0
end_at_doc = 1096 # this is not sustainable if you don't know how many documents you have in the bucket, how do you make the calculations? having many replicas of the same notebook seems completely unsustainable

similarity_threshold = 0
search_results_limit = 500
transformer_name = 'xlm-r-bert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(transformer_name)

# bucket_name = 'wri-nlp-policy'

# language, filename, prefix = adjust_for_language("Spanish")
# sentences = load_all_sentences(language, s3, bucket_name, init_at_doc, end_at_doc, prefix )

In [85]:
# Calculate and store query embeddings
query_embeddings = dict(zip(queries, [model.encode(query.lower(), show_progress_bar=False) for query in queries]))

In [95]:
# For each sentence, calculate its embedding, and store the similarity
query_similarities = defaultdict(list)
i = 0
for sent_id, sent_map in s3_client.load_sentences("english", init_at_doc, end_at_doc):
    sentence_embedding = model.encode(sent_map['text'].lower(), show_progress_bar=False)
    i += 1
    if i % 100 == 0:
        print(i)
    
    for query_text, query_embedding in query_embeddings.items():
        score = round(1 - distance.cosine(sentence_embedding, query_embedding), 4)
        if score > similarity_threshold:
            query_similarities[query_text].append([sent_id, score, sent_map['text']])
    
    if i == 15:
        break

In [97]:
# Build the query with similar sentences table for each query!
#### TODO: Sort results by similarity score -- Is this necessary if we're sorting it later anyway?
for query in query_similarities:
    query_similarities[query] = sorted(query_similarities[query], key = lambda x : x[1], reverse=True)

col_headers = ["sentence_id", "similarity_score", "text"]
query_sim_dfs = {}
for i, query in enumerate(query_similarities.keys()):
    query_sim_dfs.append(pd.DataFrame(query_similarities[query], columns=col_headers).head(search_results_limit))
# save_results_as_separate_csv(query_similarities, queries, init_at_doc, search_results_limit, aws_id, aws_secret)

In [103]:
query_sim_dfs[0]

,sentence_id,similarity_score,text
0,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_8,0.3686,SUPPLEMENTARY INFORMATION: Background The USGS...
1,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_5,0.3572,GIPSA also has determined that making the expo...
2,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_14,0.3413,GIPSA has found that transactions involving hi...
3,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_1,0.3260,   The Code of Federal Regulations is sold by ...
4,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_7,0.2987,FOR FURTHER INFORMATION CONTACT: Thomas C. O'C...
5,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_9,0.2596,Current waivers from the official inspection a...
6,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_0,0.2545,"Federal Register, Volume 76 Issue 146 (Friday,..."
7,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_3,0.1875,   ===========================================...
8,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_13,0.1752,As the high quality specialty grain market has...
9,0002304f1f671ea916ae0a1f784484eb4874ceaa_sent_4,0.1741,----------------------------------------------...


In [105]:
# Define queries... AGAIN?
path = "tasks/data_augmentation/input/"
filename = "English_queries.xlsx"
file = path + filename
df = pd.read_excel(file, engine='openpyxl', sheet_name = "Hoja1", usecols = "A:C")

policy_instrument = {}
for index, row in df.iterrows():
    if row['Policy instrument'] in policy_instrument:
        policy_instrument[row['Policy instrument']] += 1
    else:
        policy_instrument[row['Policy instrument']] = 1

In [106]:
policy_instrument

{'Direct payment': 9,
 'Fine': 7,
 'Loan': 10,
 'Supplies': 4,
 'Tax benefit': 11,
 'Technical assistance': 9}

In [ ]:
dfs.append(export_list_as_excel(file, lil_bro, item))

# Things to keep track of:
- Query text
- Query number
- Label

In [65]:
def read_csv_from_s3(s3_object, columns):
    pre_labeled =[]
    try:
        for row in csv.DictReader(codecs.getreader("utf-8")(s3_object.get()['Body'])):
            list_row = []
            for column in columns:
                list_row.append(row[column])
            pre_labeled.append(list_row)
#             print(list_row[1])
    except Exception as e:
        print(f"Problem with the file {obj.key}")
        print(e)
        pass
    return pre_labeled

def export_list_as_excel(file, list_to_save, name_of_sheet):
    # We are using excel here because we want to have 1 file that contains multiple worksheets, 1 per each label... but this may not be compatible for someone that doesn't have excel...
    df = pd.DataFrame(list_to_save)
    book = load_workbook(file)
    writer = pd.ExcelWriter(file, engine='openpyxl', mode = "a")
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
#     if name_of_sheet not in writer.sheets:
#         df.to_excel(writer, sheet_name = name_of_sheet, index=False, header=False)
#     else:
#         df.to_excel(writer, sheet_name = name_of_sheet, startrow=writer.sheets[name_of_sheet].max_row, index=False, header=False)
#     writer.save()
    return writer

# WHAT IS THIS LOL
def merge_dicts(old_dict, new_dict):
    # This function essentially updates the sentence triplet in the old dict with the triplet from the new dict if it has a higher rank than the old one. There's probably a better way of doing this. 
    for key, value in new_dict.items():
        # if the current sentence id is in the old_dict already and the rank of the current sentence is larger than the rank of the sentence already stored
        if key in old_dict and value[0] < old_dict[key][0]:
#             print("\n\n *** ", value[0], " *** ", old_dict[key][0], " ***\n\n")
            old_dict[key] = value
        elif key not in old_dict:
            old_dict[key] = value
    return old_dict

def dict_to_sorted_list(dictionary):
    llista = []
    for value in dictionary.values():
        llista.append(value)
    llista_ordenada = sorted(llista, key=lambda x: x[0])
    return llista_ordenada

In [107]:
path = "tasks/data_augmentation/output/"
filename = "pre_labeled_English.xlsx"
file = path + filename
bucket = "wri-nlp-policy"

dfs = []
bro = []
columns = ["sentence_id", "similarity_score", "text"]

####### TODO: There's something inefficient about this, we should look more in depth into it
for i, obj in enumerate(s3_client.s3.Bucket(bucket).objects.all().filter(Prefix="english_documents/assisted_labeling/")):
    print("i =", i)
    if not obj.key.endswith("/") and i < 3:
        for item in policy_instrument:
            if item in obj.key:
                print(f"Instrument: {item}")
                print(f"Obj key: {obj.key}")
                lil_bro = read_csv_from_s3(obj, columns)
                bro.append(lil_bro)
                dfs.append(export_list_as_excel(file, lil_bro, item))
#                 export_list_as_excel(file, lil_bro, item)
    
    if i == 2:
        break

i = 0
i = 1
Instrument: Direct payment
Obj key: english_documents/assisted_labeling/query_Direct payment_0_results_0.csv
i = 2
Instrument: Direct payment
Obj key: english_documents/assisted_labeling/query_Direct payment_0_results_11494.csv


In [53]:
exc = pd.read_excel(dfs[0])


In [54]:
exc.head()

,0,1091b796973f9a80ddad004b5673e8b89495b711_sent_207,0.7997,Data furnished by the applicants will be used to determine eligibility for program benefits.
0,0,b50af1454ce0ddbc85d587749339c16617aa707f_sent_185,0.8215,"Reasonable activity fees or registration fees,..."
1,0,da311c91fc18b8cb87a181abe8d65e113543e816_sent_...,0.8103,Recipients and subrecipients will be permitted...
2,0,e6a60f93da82e6d670e0d725e041ffedf3b0799c_sent_873,0.8487,The costs of these services may be allocated o...
3,0,21773f4fe9d5bec19607eb274d4a33af1d051894_sent_34,0.8046,"This could entail generating supporting data, ..."
4,0,3309c55baf714c585501a4fbc4b0a8d17aa447af_sent_977,0.8226,The costs of monitoring recipients and subreci...


In [110]:
path = "tasks/data_augmentation/output/"
filename = "pre_labeled_English_ready_short.xlsx"
file = path + filename


Dictionaries = {"Direct payment" : {}, "Fine" : {}, "Loan" : {}, "Supplies" : {}, "Tax benefit" : {}, "Technical assistance" : {}}
# Queries = {"Direct payment" : 0, "Fine" : 0, "Loan" : 0, "Supplies" : 0, "Tax benefit" : 0, "Technical assistance" : 0}


columns = ["sentence_id", "similarity_score", "text"]
for i, obj in enumerate(s3_client.s3.Bucket(bucket).objects.all().filter(Prefix="english_documents/assisted_labeling/")):
    if not obj.key.endswith("/"):# and i < 25
#         item = obj.key.split("_")[3] # 
        query_number = int(obj.key.split("_")[4]) 
#         print(obj.key.split("_"))
#         print(item, query_number)
        if query_number in Dictionaries[item]:
            new_list = read_csv_from_s3(obj, columns)
            Dictionaries[item][query_number] += new_list
        else:
            Dictionaries[item][query_number] = []
            new_list = read_csv_from_s3(obj, columns)
            Dictionaries[item][query_number] += new_list
            
    if i == 15:
        break

In [122]:
Dictionaries['Direct payment'][0][0]

['1091b796973f9a80ddad004b5673e8b89495b711_sent_207',
 '0.7997',
 'Data furnished by the applicants will be used to determine eligibility for program benefits.']

In [127]:
# What we're actually concerned about is this part. 
exx = []


# ALL OF THIS CAN BE DONE BETTER AGHHHHH JAJAJAJAJAJA this is gonna be fun
for instrument in Dictionaries:
    old_dict = {}
    for query in Dictionaries[instrument]: # where query is a pair {query_num: list of sents associated as [sent_id, score, sent]}
        temp_list = copy.deepcopy(Dictionaries[instrument][query])
        
        # sort the list of senteces for a given query by score 
        sorted_list = sorted(temp_list, key=lambda x: x[1], reverse=True) 
        new_dict = {}
        
        # we got 5000 sents for the first query
#         print(len(sorted_list))
   
        for i, item in enumerate(sorted_list):
            item.insert(0, i) # is this just adding a rank to a LOOP VARIABLE? LOL? like an ordering? HAHA 
            new_dict[item[1]] = item # and this is just... {sent id: [rank, sent id, score, sentence]}... but why?
              
#         print(new_dict)
        old_dict = merge_dicts(old_dict, new_dict) # Also {sent id: [rank, sent id, score, sentence]}... but it contains the sentences that have the highest ranking, so the new dict is for "cur triplets" and the old dict is for "highest ranked triplets"
#         print(old_dict.keys())
        
    final_list = dict_to_sorted_list(old_dict) # This is just a sorted list of triplets by rank
            
    exx.append(export_list_as_excel(file, final_list[0:501], instrument))

dict_keys(['e6a60f93da82e6d670e0d725e041ffedf3b0799c_sent_873', '61c584b26ad28a0b07eed5a8d60853cdb935ac1e_sent_336', '3309c55baf714c585501a4fbc4b0a8d17aa447af_sent_977', 'b50af1454ce0ddbc85d587749339c16617aa707f_sent_185', '8c6bae0072da455b9c356de8a2b386b5ba3ba372_sent_283', 'da311c91fc18b8cb87a181abe8d65e113543e816_sent_1027', '3309c55baf714c585501a4fbc4b0a8d17aa447af_sent_1070', 'da311c91fc18b8cb87a181abe8d65e113543e816_sent_723', '21773f4fe9d5bec19607eb274d4a33af1d051894_sent_34', '79a8a5373e3b4d9cf5039ba5d25574142654e71e_sent_114', '1091b796973f9a80ddad004b5673e8b89495b711_sent_207', 'cb85fae7dbe121cbbbe7059d48e168996ba478c1_sent_1304', 'cc83176eaa32cb5fa5a3c83e3c2854a93669492e_sent_447', '425e4474108951b2e9790efd3ecfc06d02f5af64_sent_386', 'affaec7bd301e4c065d0599c03618b9d3b23af9d_sent_451', '3309c55baf714c585501a4fbc4b0a8d17aa447af_sent_981', 'dc08ade8933b28d35c600099ca97463c05306661_sent_340', '63ea7aa7053ba342c4a9220ba5e65160cb62c2a3_sent_1098', '67fcc641b8cc79bcd734d2ab52e240a

KeyboardInterrupt: 

In [78]:
pd.read_excel(exx[0])

,Counter:,0,NaN,Direct Payment,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Rank,Sentence_Id,Similarity _score,Sentence,Is_policy,Is_incentive,Other_instrument,Comments
1,0,e6a60f93da82e6d670e0d725e041ffedf3b0799c_sent_873,0.8487,The costs of these services may be allocated o...,NaN,NaN,NaN,NaN
2,0,647cebbac89b19a3109019af2ba719100caabda1_sent_717,0.808,Some grant funding has been used for specific ...,NaN,NaN,NaN,NaN
3,0,b5453f0d5467b101546dcf3e83b96d3355e7dae2_sent_91,0.8473,The Bureau encourages applicants to provide ma...,NaN,NaN,NaN,NaN
4,0,0253b47b4cb1252a87e09ab3be80edea7465ba50_sent_84,0.8326,"When this funding becomes accessible, we antic...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
497,57,edab3785fbcdec3ceda46df91b1ad2eacf37036e_sent_315,0.7802,(2) Evaluation is an integral element of progr...,NaN,NaN,NaN,NaN
498,57,72226ac41f3eba5179ad372a67639965a0b4f7fd_sent_63,0.6972,The intermediary must provide matching funds a...,NaN,NaN,NaN,NaN
499,57,aeb586de1440f82ed95596c3e10513278d3119fb_sent_34,0.7933,Grant and matching funds must be utilized in a...,NaN,NaN,NaN,NaN
500,57,9a68413b1667357bc39ef4537f68953302acb07c_sent_65,0.668,Those applications receiving the highest point...,NaN,NaN,NaN,NaN
